### Model Drift Analysis: train the model and save to Model Catalog

Model Drift Analysis require two dataset containing not only the features (xi) but also the target.

It means that, in order to monitor Model's performances and detect Model drift, we need, in some way, to collect data and analyze the results in order to define the "ground truth".

In this NB I have put a prototype that can be used to **start working on Model Drift**.

The dataset used is again the Employee Attrition Data and the model is based on LightGBM (GBM) and Sklearn pipeline.

We simulate a Data Drift (adding a "shift" to some features) in order to make performances worse.

In the First Part of the NB we train a model on a reference dataset and we save the pipeline + the metrics computed on a reference validation dataset.
In the second part we reload the model (pipeline) and we re-evaluate the metrics on a new dataset.
All the  results are saved in a csv file that can be easily loaded in a DB.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import tempfile

import ads
from ads import set_auth

# to save to Model Catalog
from ads.catalog.model import ModelCatalog
from ads.common.model_metadata import UseCaseType, MetadataCustomCategory
from ads.model.framework.sklearn_model import SklearnModel

# used to serialize the pipeline
from pickle import dump, load

import lightgbm as lgb

from sklearn.metrics import classification_report
from sklearn.metrics import get_scorer, make_scorer, f1_score, roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

# added to handle with pipelines
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from ads.dataset.factory import DatasetFactory

import logging
import warnings

import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

%matplotlib inline

In [2]:
# we need ads 2.5.10 or greater
print(ads.__version__)

2.5.10


In [3]:
# set RP
set_auth(auth='resource_principal')

### First Part: train the model

In [4]:
#
# definisco le funzioni che identificano le categorie di colonne
#
def cat_cols_selector(df, target_name):
    # the input is the dataframe
    
    # cols with less than THR values are considered categoricals
    THR = 10
    
    nunique = df.nunique()
    types = df.dtypes
    
    col_list = []
    
    for col in df.columns:
        if ((types[col] == 'object') or (nunique[col] < THR)):
            # print(col)
            if col != target_name:
                col_list.append(col)
    
    return col_list

def num_cols_selector(df, target_name):
    THR = 10
    
    types = df.dtypes
    nunique = df.nunique()
    
    col_list = []
    
    for col in df.columns:
        if (types[col] != 'object') and (nunique[col] >= THR): 
            # print(col)
            if col != target_name:
                col_list.append(col)
    
    return col_list

def load_as_dataframe(path):
    ds = DatasetFactory.open(path,
                             target="Attrition").set_positive_class('Yes')

    ds_up = ds.up_sample()

    # drop unneeded columns
    cols_to_drop = ['Directs','name', 'Over18','WeeklyWorkedHours','EmployeeNumber']

    ds_used = ds_up.drop(columns=cols_to_drop)
    
    df_used = ds_used.to_pandas_dataframe()
    
    

    # train, test split (lo faccio direttamente sui dataframe)
    df_train, df_test = train_test_split(df_used, shuffle=True, test_size=0.2, random_state = 1234)

    print("# of samples in train set", df_train.shape[0])
    print("# of samples in test set", df_test.shape[0])
    
    return df_train, df_test

In [5]:
# load the dataset and do upsampling
TARGET = 'Attrition'

attrition_path = "/opt/notebooks/ads-examples/oracle_data/orcl_attrition.csv"

df_train, df_test = load_as_dataframe(attrition_path)

# uso ancora la classe dataset per fare l'upsampling

cat_cols = cat_cols_selector(df_train, TARGET)
num_cols = num_cols_selector(df_train, TARGET)

print()
print(f'Numerical columns: {num_cols} ({len(num_cols)})')
print()
print(f'Categorical columns: {cat_cols} ({len(cat_cols)})')

loop1:   0%|          | 0/4 [00:00<?, ?it/s]

# of samples in train set 1972
# of samples in test set 494

Numerical columns: ['Age', 'SalaryLevel', 'CommuteLength', 'HourlyRate', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'YearsinIndustry', 'YearsOnJob', 'YearsAtCurrentLevel', 'YearsSinceLastPromotion', 'YearsWithCurrManager'] (13)

Categorical columns: ['TravelForWork', 'JobFunction', 'EducationalLevel', 'EducationField', 'EnvironmentSatisfaction', 'Gender', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TrainingTimesLastYear', 'WorkLifeBalance'] (17)


In [6]:
#
# creo la parte Transformers per le pipeline
#

# per questo dataset non vi sono missing values
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('standard_scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)])

In [7]:
X_train, y_train = df_train.drop([TARGET], axis=1), df_train[TARGET]
X_test, y_test = df_test.drop([TARGET], axis=1), df_test[TARGET]

In [8]:
#
# definisco la pipeline completa
#
clf = Pipeline(steps=[('preprocessor', transformations),
                           ('clf', lgb.LGBMClassifier())])

In [9]:
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('standard_scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SalaryLevel',
                                                   'CommuteLength',
                                                   'HourlyRate',
                                                   'MonthlyIncome',
                                                   'MonthlyRate',
                                                   'NumCompaniesWorked',
                                                   'PercentSalaryHike',
                                                   'YearsinIndustry',
                  

### Score the Model on the test dataset

In [10]:
test_pred = clf.predict(X_test)
test_probas = clf.predict_proba(X_test)

print('Validation set result:')

roc_auc = round(roc_auc_score(y_test, test_probas[:,1]), 4)
acc = round(accuracy_score(y_test, test_pred), 4)

# this is the Object that will be saved in the Catalog
metrics = {
    "accuracy" : acc,
    "roc_auc" : roc_auc
}

print(str(metrics))

Validation set result:
{'accuracy': 0.9494, 'roc_auc': 0.9951}


#### Save metrics and model

In [11]:
# save in a file the metrics computed on the reference set
now = datetime.now().strftime('%Y-%m-%d %H:%M')

dict_ref = [{
    "ts_date": now,
    "model_name": "lgb1",
    "algorithm": "lightgbm",
    "accuracy": acc,
    "roc_auc": roc_auc
}]

df_ref = pd.DataFrame(dict_ref)

# save initial file
df_ref.to_csv("model_metrics.csv", index=None)

#### Save model to the Model Catalog

In [12]:
artifact_dir = tempfile.mkdtemp()

# with SklearnModel there is support for pipelines
sklearn_model = SklearnModel(estimator=clf, artifact_dir= artifact_dir)

In [13]:
# this is the env for runtime (don't need to upgrade ads... otherwise you would need a custom conda env)
CONDA_ENV_SLUG = "generalml_p37_cpu_v1"

sklearn_model.prepare(
    inference_conda_env=CONDA_ENV_SLUG,
    training_conda_env=CONDA_ENV_SLUG,
    use_case_type=UseCaseType.BINARY_CLASSIFICATION,
    as_onnx=False,
    X_sample=X_test,
    y_sample=y_test,
    force_overwrite=True,
)

In [14]:
sklearn_model.verify(X_test.head(5))

Start loading model.joblib from model directory /tmp/tmpkwqfsp4x ...
Model is successfully loaded.


{'prediction': [1, 1, 0, 1, 0]}

In [15]:
# add info on reference dataset used for training

ref_url = "oci://drift_input@frqap2zhtzbe/reference.csv"

sklearn_model.metadata_custom.add(key='reference dataset', value=ref_url, category=MetadataCustomCategory.TRAINING_AND_VALIDATION_DATASETS, 
                                  description='Reference dataset url. From this dataset have been extracted train/validation dataset', replace=True)

sklearn_model.metadata_custom.add(key='metrics on reference set', value=str(metrics), category=MetadataCustomCategory.PERFORMANCE, 
                                  description='Metrics evaluated on reference dataset', replace=True)

In [16]:
# check all custom metadata
sklearn_model.metadata_custom

data:
- category: Training Environment
  description: The URI of the training conda environment.
  key: CondaEnvironmentPath
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General Machine Learning
    for CPUs on Python 3.7/1.0/generalml_p37_cpu_v1
- category: Training Environment
  description: The conda environment where the model was trained.
  key: CondaEnvironment
  value: oci://service-conda-packs@id19sfcrra6z/service_pack/cpu/General Machine Learning
    for CPUs on Python 3.7/1.0/generalml_p37_cpu_v1
- category: Training Profile
  description: The model serialization format.
  key: ModelSerializationFormat
  value: joblib
- category: Training Environment
  description: The slug name of the training conda environment.
  key: SlugName
  value: generalml_p37_cpu_v1
- category: Other
  description: ''
  key: ClientLibrary
  value: ADS
- category: Performance
  description: Metrics evaluated on reference dataset
  key: metrics on reference set
  value: '{''accuracy

In [17]:
# save to the Model Catalog

MODEL_NAME = "employee-attrition-lgbm05"
model_id = sklearn_model.save(display_name=MODEL_NAME)

print(f"Model id in Model Catalog is {model_id}")

Start loading model.joblib from model directory /tmp/tmpkwqfsp4x ...
Model is successfully loaded.
['input_schema.json', 'score.py', 'model.joblib', 'runtime.yaml', 'output_schema.json']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

artifact:/tmp/saved_model_3743bb7d-09ca-450e-96e7-301b7611395b.zip
Model id in Model Catalog is ocid1.datasciencemodel.oc1.eu-frankfurt-1.amaaaaaangencdyasojemavtoshdggls4rg27i2qctcin6xz3yi3yevhnaha


### Second Part: analysis on a new dataset

see: Notebook part 2